# 함수, 반복자, 제네레이터

함수형 프로그래밍의 핵심은 순수 함수를 사용해 입력 정의력의 값을 출력 치역의 값으로 바꾸는 것이다. 순수 함수에는 부수 효과가 없으며, 이러한 순수 함수를 파이썬으로 구현하는 것은 상대적으로 쉽다.

부수 효과가 없다는 것은 계산에 필요한 상태를 유지하기 윈한 변수 대입에 따른 의존 관계를 줄일 수 있다는 의미다. 파이썬 언어에서 대입문을 아예 없앨 수는 없지만, 상태가 있는 객체에 대한 의존성을 줄일 수는 있다. 

제네레이터와 제네레이터 식이 객체의 컬렉션을 가지고 작업하기 위한 방법이기 때문에 이에 대해 살펴본다. 모든 제네레이터 식을 재귀로 바꿀 때는 몇 가지 주변적인 문제가 생긴다. 파이썬은 재귀 깊이에 제한을 두며, 자동으로 TCO하지 않는다. 우리는 제네레이터 식을 사용해 직접 재귀를 최적화해야만 한다.

함수형 파이썬을 작성하려면 튜플과 변경할 수 있는 컬렉션에 초점을 맞출 필요가 있다.

## 순수 함수 작성하기
